# Titanic - Machine Learning from Disaster

https://www.kaggle.com/competitions/titanic

**Goal:** Predict survival on the Titanic (binary classification)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)

## 1. Load Data

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(f"Train: {train.shape}, Test: {test.shape}")
train.head()

Train: (891, 12), Test: (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# Missing values
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 2. Quick EDA

In [5]:
# Survival rate
train['Survived'].value_counts(normalize=True)

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64

In [6]:
# Survival by Sex
train.groupby('Sex')['Survived'].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [7]:
# Survival by Pclass
train.groupby('Pclass')['Survived'].mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

## 3. Feature Engineering & Preprocessing

In [12]:
def preprocess(df):
    """Basic preprocessing for Titanic data."""
    result = df.copy()

    # Fill missing values FIRST
    result['Age'] = result['Age'].fillna(result['Age'].median())
    result['Embarked'] = result['Embarked'].fillna(result['Embarked'].mode()[0])
    result['Fare'] = result['Fare'].fillna(result['Fare'].median())

    # Title
    result['Title'] = result['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
    result['Title'] = result['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr',
                                                'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    result['Title'] = result['Title'].replace(['Mlle', 'Ms'], 'Miss')
    result['Title'] = result['Title'].replace('Mme', 'Mrs')
    result['Title'] = result['Title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4})
    result['Title'] = result['Title'].fillna(0)

    # Family
    result['FamilySize'] = result['SibSp'] + result['Parch'] + 1
    result['IsAlone'] = (result['FamilySize'] == 1).astype(int)

    # Encode
    result['Sex'] = result['Sex'].map({'male': 0, 'female': 1})
    result['Embarked'] = result['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    
    return result

In [13]:
train_processed = preprocess(train)
test_processed = preprocess(test)

# Features for model
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title', 'FamilySize', 'IsAlone']

X = train_processed[features]
y = train_processed['Survived']
X_test = test_processed[features]

print(f"X: {X.shape}, y: {y.shape}, X_test: {X_test.shape}")

X: (891, 10), y: (891,), X_test: (418, 10)


## 4. Model Training

In [14]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# Cross-validation
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f"CV Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")

CV Accuracy: 0.8283 (+/- 0.0163)


In [15]:
# Train on full data
model.fit(X, y)

# Feature importance
pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

,feature,importance
7,Title,0.313738
1,Sex,0.242125
5,Fare,0.126250
0,Pclass,0.112853
8,FamilySize,0.067157
2,Age,0.066501
3,SibSp,0.028068
6,Embarked,0.017939
4,Parch,0.013765
9,IsAlone,0.011604


## 5. Create Submission

In [16]:
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})

submission.to_csv('submission.csv', index=False)
print(f"Submission saved: {submission.shape}")
submission.head()

Submission saved: (418, 2)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
